In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time

In [2]:
characters = pd.read_csv('merged.csv')

In [3]:
characters.head()

,rank,name,anime,favorites,url,description,image,Age,Birthday,Nationality,Height,Occupation,image.1
0,1,"Lamperouge, Lelouch(ルルーシュ・ランペルージ)",Code Geass: Hangyaku no LelouchCode Geass: Han...,173552,https://myanimelist.net/character/417/Lelouch_...,"Age: 17 (first season), 18 (second season) \n ...",NaN,"17 (first season), 18 (second season)","December 5, 1999 a.t.b. (Sagittarius)",Britannia,"178 cm (5'10"")","Student, Terrorist, Knightmare Pilot, Politici...",Error processing https://myanimelist.net/chara...
1,2,"Monkey D., Luffy(モンキー・D・ルフィ)",One PieceOne Piece Movie 01One Piece Movie 02:...,143840,https://myanimelist.net/character/40/Luffy_Mon...,"Age: 17; 19 \n Birthdate: May 5, Taurus \n Hei...",NaN,17; 19,"May 5, Taurus",NaN,172 cm; 174 cm,NaN,https://cdn.myanimelist.net/images/characters/...
2,3,Levi(リヴァイ),Shingeki no Kyojin: Kuinaki SentakuShingeki no...,143364,https://myanimelist.net/character/45627/Levi,"Birthday: December 25 \n Height: 160 cm (5'3"")...",NaN,NaN,NaN,NaN,"160 cm (5'3"")",NaN,https://cdn.myanimelist.net/images/characters/...
3,4,"Lawliet, L(エル ローライト)",Death NoteDeath Note: Rewrite,128375,https://myanimelist.net/character/71/L_Lawliet,"Birthday: October 31, 1979 (1982 in anime) \n ...",NaN,NaN,NaN,NaN,"179 cm (5'10"")",NaN,Error processing https://myanimelist.net/chara...
4,5,"Roronoa, Zoro(ロロノア・ゾロ)",One PieceOne Piece Movie 01One Piece Movie 02:...,113243,https://myanimelist.net/character/62/Zoro_Roronoa,Error processing https://myanimelist.net/chara...,NaN,NaN,NaN,NaN,NaN,NaN,https://cdn.myanimelist.net/images/characters/...


In [4]:
description_errors = characters['description'].str.startswith('Error processing').sum()

image_errors = characters['image.1'].str.startswith('Error processing').sum()

total_errors = (
    characters['description'].str.startswith('Error processing') |
    characters['image.1'].str.startswith('Error processing')
).sum()

print(f"Number of 'Error processing' in description: {description_errors}")
print(f"Number of 'Error processing' in image.1: {image_errors}")
print(f"Total rows with 'Error processing' in either column: {total_errors}")


Number of 'Error processing' in description: 89
Number of 'Error processing' in image.1: 96
Total rows with 'Error processing' in either column: 177
